In [1]:
import scipy.stats as st
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from pandas.io.json import json_normalize


Bad key "text.kerning_factor" on line 4 in
/Users/carolinecharrier/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df = pd.read_excel('output/df3.xlsx', sheet_name='Sheet1')
del df['Unnamed: 0']
df

,country,year_survey,quantile,income,gdpppp,country_name,gini,IGEincome
0,ALB,2008,1,728.89795,7297.00000,Albanie,30.462429,0.815874
1,ALB,2008,2,916.66235,7297.00000,Albanie,30.462429,0.815874
2,ALB,2008,3,1010.91600,7297.00000,Albanie,30.462429,0.815874
3,ALB,2008,4,1086.90780,7297.00000,Albanie,30.462429,0.815874
4,ALB,2008,5,1132.69970,7297.00000,Albanie,30.462429,0.815874
...,...,...,...,...,...,...,...,...
11595,COD,2008,97,911.78340,303.19305,"Congo, République démocratique du",44.399690,0.500000
11596,COD,2008,98,1057.80740,303.19305,"Congo, République démocratique du",44.399690,0.500000
11597,COD,2008,99,1286.60290,303.19305,"Congo, République démocratique du",44.399690,0.500000
11598,COD,2008,100,2243.12260,303.19305,"Congo, République démocratique du",44.399690,0.500000


In [3]:
# Clonage des lignes 
df_clone = pd.concat([df] * 500)
df_clone.sort_values(by=['country', 'quantile'], inplace=True)

In [4]:
# Mise en forme 
df_clone.rename(columns={'country_name':'Pays','quantile': 'c_i_child', 'income': 'y_child', 'gini': 'Gj', 'IGEincome': 'pj'}, inplace=True)
df_clone = df_clone[['Pays', 'c_i_child', 'y_child', 'Gj', 'pj']]
df_clone.head(3)

,Pays,c_i_child,y_child,Gj,pj
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874


In [5]:
def generate_incomes(n, pj):
    # On génère les revenus des parents (exprimés en logs) selon une loi normale.
    # La moyenne et variance n'ont aucune incidence sur le résultat final (ie. sur le caclul de la classe de revenu)
    ln_y_parent = st.norm(0,1).rvs(size=n)
    # Génération d'une réalisation du terme d'erreur epsilon
    residues = st.norm(0,1).rvs(size=n)
    return np.exp(pj*ln_y_parent + residues), np.exp(ln_y_parent)

def quantiles(l, nb_quantiles):
    size = len(l)
    #taille de l'échantillon
    l_sorted = l.copy()
    l_sorted = l_sorted.sort_values()
    #tri des valeurs
    quantiles = np.round(np.arange(1, nb_quantiles+1, nb_quantiles/size) -0.5 +1./size)
    #création des quantiles
    q_dict = {a:int(b) for a,b in zip(l_sorted,quantiles)}
    #association des quantiles aux y_child, y_parents
    return pd.Series([q_dict[e] for e in l])
def compute_quantiles(y_child, y_parents, nb_quantiles):
    y_child = pd.Series(y_child)
    y_parents = pd.Series(y_parents)
    #conversion en Series
    c_i_child = quantiles(y_child, nb_quantiles)
    c_i_parent = quantiles(y_parents, nb_quantiles)
    #création des quantiles associés aux y_child et y_parents
    sample = pd.concat([y_child, y_parents, c_i_child, c_i_parent], axis=1)
    #mise bout a bout
    sample.columns = ["y_child", "y_parents", "c_i_child","c_i_parent"]
    #renommage des colonnes
    return sample

def distribution(counts, nb_quantiles):
    distrib = []
    total = counts["counts"].sum()
    #nb total de cas = 100000
    
    if total == 0 :
        return [0] * nb_quantiles
    
    for q_p in range(1, nb_quantiles+1): # pour chaque quantile_parent entre 1 et 100
        subset = counts[counts.c_i_parent == q_p] #crée un sous-échantillon de ce quantile
        if len(subset): #si le sous-échantillon n'est pas vide
            nb = subset["counts"].values[0] #la distribution est calculée
            distrib += [nb / total] #ramenée en portion du total et ajouté a la variable
        else:
            distrib += [0]
    return distrib

def conditional_distributions(sample, nb_quantiles):
    counts = sample.groupby(["c_i_child","c_i_parent"]).apply(len)
    counts = counts.reset_index()
    counts.columns = ["c_i_child","c_i_parent","counts"]
    
    
    mat = []
    for child_quantile in np.arange(nb_quantiles)+1: # pour chaque child_quantile entre 1 et 100
        subset = counts[counts.c_i_child == child_quantile]#crée un subset de ce quantile
        mat += [distribution(subset, nb_quantiles)]#applique distribution() au subset
        #et rajoute le résultat dans "mat"
    return np.array(mat)

def condi_pays(n, nb_quantiles, pj_pays):
    y_child,y_parents = generate_incomes(n,pj = pj_pays)
    sample = compute_quantiles(y_child,y_parents,nb_quantiles)
    cd = conditional_distributions(sample,nb_quantiles)
    return cd

In [6]:
df_clone

,Pays,c_i_child,y_child,Gj,pj
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
...,...,...,...,...,...
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000


In [7]:
# Read CSV updated
kta = pd.read_excel('kt-updated.xlsx', sheet_name='kt-updated')
kta = kta.set_index(['country','ci_child'])

In [8]:
kta

ci_parent_1  ci_parent_2  ci_parent_3  ci_parent_4  \
country ci_child                                                       
ALB     0               0.203        0.134        0.077        0.068   
        1               0.119        0.073        0.064        0.058   
        2               0.076        0.057        0.059        0.057   
        3               0.069        0.053        0.041        0.045   
        4               0.054        0.044        0.050        0.047   
...                       ...          ...          ...          ...   
COD     95              0.000        0.001        0.001        0.000   
        96              0.000        0.000        0.002        0.001   
        97              0.000        0.000        0.000        0.000   
        98              0.000        0.001        0.000        0.000   
        99              0.000        0.000        0.000        0.000   

                  ci_parent_5  ci_parent_6  ci_parent_7  ci_parent_8  \
country ci_child                                                       
ALB     0               0.045        0.045        0.039        0.034   
        1               0.057        0.046        0.046        0.033   
        2               0.037        0.042        0.034        0.036   
        3               0.042        0.051        0.052        0.045   
        4               0.046        0.038        0.030        0.035   
...                       ...          ...          ...          ...   
COD     95              0.001        0.001        0.001        0.001   
        96              0.001        0.001        0.001        0.002   
        97              0.000        0.001        0.002        0.003   
        98              0.001        0.001        0.000        0.001   
        99              0.000        0.000        0.000        0.001   

                  ci_parent_9  ci_parent_10  ...  ci_parent_91  ci_parent_92  \
country ci_child                             ...                               
ALB     0               0.027         0.031  ...         0.000         0.001   
        1               0.032         0.026  ...         0.000         0.000   
        2               0.027         0.031  ...         0.000         0.000   
        3               0.028         0.024  ...         0.000         0.000   
        4               0.041         0.026  ...         0.000         0.000   
...                       ...           ...  ...           ...           ...   
COD     95              0.002         0.005  ...         0.014         0.028   
        96              0.002         0.003  ...         0.028         0.030   
        97              0.001         0.000  ...         0.024         0.020   
        98              0.001         0.001  ...         0.021         0.022   
        99              0.000         0.001  ...         0.022         0.031   

                  ci_parent_93  ci_parent_94  ci_parent_95  ci_parent_96  \
country ci_child                                                           
ALB     0                0.000         0.000         0.000         0.000   
        1                0.000         0.000         0.001         0.000   
        2                0.000         0.000         0.000         0.000   
        3                0.000         0.000         0.000         0.000   
        4                0.000         0.000         0.000         0.000   
...                        ...           ...           ...           ...   
COD     95               0.023         0.021         0.029         0.023   
        96               0.032         0.021         0.022         0.033   
        97               0.031         0.031         0.033         0.026   
        98               0.029         0.026         0.027         0.037   
        99               0.033         0.029         0.032         0.036   

                  ci_parent_97  ci_parent_98  ci_parent_99  ci_parent_100  
country ci_child                      

In [9]:
quantiles_associes=[]
for ind,row in kta.iterrows():
    proba = kta.values
    for z in range(100):#pour chaque quantile enfant:
        probabilites_quantile_z=proba[z]
        quantile_associe=np.random.choice(1, 500, p=probabilites_quantile_z)
        quantiles_associes+=[quantile_associe]
        
quantiles_associes_merged=[j for i in quantiles_associes for j in i]

In [10]:
grr = pd.DataFrame(quantiles_associes_merged) 
grr

,0
0,7
1,5
2,1
3,13
4,18
...,...
579999995,93
579999996,94
579999997,78
579999998,85


In [11]:
df_clone

,Pays,c_i_child,y_child,Gj,pj
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
0,Albanie,1,728.89795,30.462429,0.815874
...,...,...,...,...,...
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000
11498,Afrique du Sud,100,82408.55000,66.977850,0.677000


In [12]:
df_clone = df_clone.reset_index()

In [ ]:
final = pd.concat([df_clone,grr],axis=1)
final 